En esta clase extraemos el título del csv

In [78]:
import re

class ExtractTitleCSV():
    
    __regExp = "([a-zA-Z0-9]+.csv)"
    __regExpSust = '[0-9]+(.csv)'
    
    def __init__(self, link):
        self.__link = link.lower()
        
    def getTitle(self):
        title = self.__setTitleWithCSV()
        title = self.__setTitleOffCSV(title)
        return title
    
    def __setTitleWithCSV(self):
        print(re.findall(self.__regExp, self.__link))
        return re.findall(self.__regExp, self.__link)[0]
    
    def __setTitleOffCSV(self, title):
        return re.sub(self.__regExpSust, "", title)

En esta clase lo que hacemos es seleccionar el rango de datos a tomar

In [79]:
import pandas as pd

class AssingDataEntryRange():
    
    def __init__(self, link):
        self.__link = link
        self.__df = pd.read_csv(link, encoding ='utf-8', delimiter=',', header = None)
        self.__lengthDataFlow = len(self.__df.count(axis=1)) - 1
    
    def mostrarLink(self):
        return self.__link
    
    def getDF(self):
        self.__orderDataFrame()
        return self.__df
    
    def showAllData(self):
        return (
            self.__link,
            self.__lengthDataFlow,
            self.__df
        )
    
    def __orderDataFrame(self):
        self.__deleteTrashDataTail()
        self.__deleteTrashDataHeader()
        self.__dropAllDataMissing()
        self.__resetIndexData()
        self.__df.columns = range(self.__df.columns.size)
        self.__lengthDataFlow = len(self.__df.count(axis=1)) - 1
    
    def __dropAllDataMissing(self):
        self.__df.dropna(axis = 1, how = 'all', inplace = True)
        self.__df.dropna(axis = 0, how = 'all', inplace = True)
    
    def __deleteTrashDataTail(self):
        cicloWhile = self.__lengthDataFlow
        while cicloWhile > 0:
            if pd.isna(self.__df[2][cicloWhile]) == True:
                self.__df.drop([cicloWhile], axis=0, inplace = True)
                cicloWhile = cicloWhile - 1
            else:
                break
    
    def __deleteTrashDataHeader(self):
        self.__df.drop([0, 1, 2], axis=0, inplace = True)
    
    def __resetIndexData(self):
        self.__df.reset_index(drop = True, inplace = True)
        
    

En esta clase se nos devolvera la cantidad de subTitulos que hay

In [80]:
import pandas as pd
import re

class SetSubtitlesColumns():
    
    def __init__(self, df):
        self.__df = df
        self.__subtitlesColumns = 0
    
    def getSubtitlesColumns(self):
        self.__setSubTitlesColumns()
        return self.__subtitlesColumns
    
    def __setSubTitlesColumns(self):
        nanValue = self.__checkNaNvalue()
        if nanValue == True:
            self.__nanTitlePrincipal()
        else:
            self.__notNanTitlePrincipal()
        
    def __checkNaNvalue(self):
        if pd.isna(self.__df[0][self.__subtitlesColumns]) == True:  
            self.__subtitlesColumns = self.__subtitlesColumns + 1
            return True
        if self.__subtitlesColumns == 0:
            self.__subtitlesColumns = self.__subtitlesColumns + 1
        return False
    
    def __nanTitlePrincipal(self):
        nanValue = self.__checkNaNvalue()
        while nanValue == True:
            nanValue = self.__checkNaNvalue()
            
    def __notNanTitlePrincipal(self):
        nanValue = self.__checkNaNvalue()
        if nanValue == False:
            self.__subtitlesColumns = 1
            return
        while nanValue == True:
            nanValue = self.__checkNaNvalue()
        

En esta clase transformamos los datos de la cabecera

In [81]:
import pandas as pd
import re

class TransformHeadData():
    
    __regExp = "[a-zA-Z0-9ñÑáéíóúÁÉÍÓÚ\s]+[^\/\s\n]+"
    
    def __init__(self, df, subTitlesColumns):
        self.__df = df
        self.__subtitlesColumns = subTitlesColumns
        self.__lengthDF = len(df.columns)
        
    def getDF(self):
        self.__transformHeadData()
        return self.__df
    
    def __transformHeadData(self):
        for index in range(self.__subtitlesColumns):
            self.__editHeader(index)
            self.__addDataTitle(index)
            
    def __editHeader(self, column):
        x = 0
        while x < self.__lengthDF:
            if pd.isna(self.__df[x][column]) != True:
                df[x][column] = re.findall(self.__regExp, self.__df[x][column])[0]
            x = x + 1
    
    def __addDataTitle(self, column):
        x = 0
        while x < self.__lengthDF:
            if pd.isna(self.__df[x][column]) == True and x == 0:
                x = x + 1
                continue
            if pd.isna(self.__df[x][column]) == True:
                self.__df[x][column] = self.__df[x - 1][column]
            x = x + 1


En esta clase se empieza a sacar lo que es el formato Json

In [82]:
import json

class CreateJson():
    
    def __init__(self, df, columns, title, aditionalData):
        self.__df = df
        self.__columns = columns
        self.__title = title
        self.__aditionalData = aditionalData
        self.__dic = []
        self.__lengthDFcolumns = len(df.columns)
        
    def getDataLessDF(self):
        return f"columnas: {self.__columns}\nTitle: {self.__title}\nAditional Data: {self.__aditionalData}\n"
    
    def getJsonData(self):
        for index in range (1, self.__lengthDFcolumns):
            self.__dic.append(self.__assignFormatJson(index)) 
        return self.__dic
    
    def __assignFormatJson(self, index):
        data = self.__df.loc[:, [0, index]]
        jsonSubtitles = self.__generateJsonDataTitles(data, index)
        indexTitle = data[0][0]
        diccionary = self.__setJsonData(data, jsonSubtitles, indexTitle, index)
        return diccionary
    
    def __generateJsonDataTitles(self, data, position):
        jsonData = dict()
        for index in range(self.__columns):
            title = f"Titulo {index + 1}"
            jsonData[title] = data[position][index]
        return jsonData
    
    def __setJsonData(self, data, jsonSubtitle, indexTitle, index):
        diccionario = []
        for position in range(self.__columns, len(data)):
            jsonData = dict()
            datos = data.loc[position]
            jsonData[indexTitle] = datos[0]
            jsonData["Datos"] = datos[index]
            jsonData["sector"] = self.__aditionalData['sector']
            jsonData["subSector"] = self.__aditionalData['subSector']
            jsonData["categoria"] = self.__title
            jsonData["Institucion"] = self.__aditionalData['Institucion']
            jsonData.update(jsonSubtitle)
            diccionario.append(jsonData)
        return diccionario
    

In [91]:
links = [
    "https://datosabiertos.impi.gob.mx/IMPICifras/Invenciones02.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Promocion05.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Invenciones16.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Proteccion04.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Proteccion03.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Proteccion01.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Promocion06.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Promocion02.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Promocion01.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Regionales01.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/PIB01.CSV",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Marcas12.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Marcas05.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Marcas01.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Invenciones15.csv",
    "https://datosabiertos.impi.gob.mx/IMPICifras/Invenciones01.csv",
]

aditionalData = {
    "sector": "INNOVACIÓN",
    "subSector": "PATENTES",
    "Institucion": "IMPI",
}

orderData = AssingDataEntryRange(links[12])
df = orderData.getDF()
link = orderData.mostrarLink()
title = ExtractTitleCSV(link)

In [92]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,País/\nCountry,Acumulado / \nAccumulate,1993,1994,1995,1996,1997,1998,1999,2000,...,1T/2021,2T/2021,3T/2021,4T/2021,Anual\n2021,1T/2022,2T/2022,3T/2022,4T/2022,Anual\n2022
1,Afganistán,2,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,Albania,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,Alemania,"47,193",398,769,"1,090",876,820,828,"1,305","1,679",...,112,163,124,94,493,86,94,119,114,413
4,Andorra,1,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,1,-,1


In [93]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
205,Zimbabwe,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
206,Apátrida,22,-,-,-,-,-,-,-,-,...,1,1,4,-,6,1,2,1,1,5
207,Otros,"127,692","1,146","2,070","2,063","1,603","1,602","1,697","2,588","3,035",...,1,-,-,-,1,6,-,2,-,8
208,Protocolo de Madrid,"69,360",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"5,436","10,789","7,062","4,733","28,020","7,655","12,941","12,204","4,208","37,008"
209,Suma,"2,025,730","20,893","33,988","29,954","25,983","27,821","28,362","40,321","45,483",...,"34,702","40,110","39,276","33,263","147,351","37,692","36,832","35,511","28,915","138,950"


In [94]:
titleCSV = title.getTitle()
print(titleCSV)

['marcas05.csv']
marcas


In [95]:
subtitlesHead = SetSubtitlesColumns(df)
columns = subtitlesHead.getSubtitlesColumns()
print(columns)

1


In [96]:
headTransform = TransformHeadData(df, columns)
dfTransform = headTransform.getDF()
dfTransform.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,País,Acumulado,1993,1994,1995,1996,1997,1998,1999,2000,...,1T,2T,3T,4T,Anual\n2021,1T,2T,3T,4T,Anual\n2022
1,Afganistán,2,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,Albania,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,Alemania,"47,193",398,769,"1,090",876,820,828,"1,305","1,679",...,112,163,124,94,493,86,94,119,114,413
4,Andorra,1,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,1,-,1


In [97]:
jsonFormat = CreateJson(dfTransform, columns, titleCSV, aditionalData)
print(jsonFormat.getDataLessDF())

columnas: 1
Title: marcas
Aditional Data: {'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'Institucion': 'IMPI'}



In [98]:
diccionario= jsonFormat.getJsonData()
a = 1
print(diccionario)
for dic in diccionario:
    print("Ciclo: ", a)
    for dat in dic:
        print()
        print(dat)
        print()
    a += 1

[[{'País': 'Afganistán', 'Datos': '2', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Albania', 'Datos': '-', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Alemania', 'Datos': '47,193', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Andorra', 'Datos': '1', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Angola', 'Datos': '-', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Antigua y Barbuda', 'Datos': '10', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Arabia Saudita', 'Datos': '28', 'sector

In [99]:
print(diccionario[0])

[{'País': 'Afganistán', 'Datos': '2', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Albania', 'Datos': '-', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Alemania', 'Datos': '47,193', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Andorra', 'Datos': '1', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Angola', 'Datos': '-', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Antigua y Barbuda', 'Datos': '10', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Arabia Saudita', 'Datos': '28', 'sector'

In [100]:
dataJSON = diccionario[0]

In [101]:
print(dataJSON)

[{'País': 'Afganistán', 'Datos': '2', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Albania', 'Datos': '-', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Alemania', 'Datos': '47,193', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Andorra', 'Datos': '1', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Angola', 'Datos': '-', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Antigua y Barbuda', 'Datos': '10', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Arabia Saudita', 'Datos': '28', 'sector'

In [102]:
dataLoad = json.dumps(dataJSON)
jsonFormat = json.loads(dataLoad)
print(jsonFormat[0])

{'País': 'Afganistán', 'Datos': '2', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}


In [103]:
dfNew = pd.DataFrame(jsonFormat)
dfNew.head()

,País,Datos,sector,subSector,categoria,Institucion,Titulo 1
0,Afganistán,2,INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado
1,Albania,-,INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado
2,Alemania,"47,193",INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado
3,Andorra,1,INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado
4,Angola,-,INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado


In [105]:
dfNew.tail()

,País,Datos,sector,subSector,categoria,Institucion,Titulo 1
204,Zimbabwe,-,INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado
205,Apátrida,22,INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado
206,Otros,"127,692",INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado
207,Protocolo de Madrid,"69,360",INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado
208,Suma,"2,025,730",INNOVACIÓN,PATENTES,marcas,IMPI,Acumulado


In [104]:
data_dict = dfNew.to_dict("records")
print(data_dict)

[{'País': 'Afganistán', 'Datos': '2', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Albania', 'Datos': '-', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Alemania', 'Datos': '47,193', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Andorra', 'Datos': '1', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Angola', 'Datos': '-', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Antigua y Barbuda', 'Datos': '10', 'sector': 'INNOVACIÓN', 'subSector': 'PATENTES', 'categoria': 'marcas', 'Institucion': 'IMPI', 'Titulo 1': 'Acumulado'}, {'País': 'Arabia Saudita', 'Datos': '28', 'sector'